In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('hr_data.csv')

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates()

In [ ]:
data.isnull().sum()

In [ ]:
data.info()


In [ ]:
data.describe()

In [ ]:
data.nunique()

In [ ]:
data['number_project'].unique()

In [ ]:
data['number_project'].value_counts()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot('number_project', data = data)
plt.show()

In [ ]:
data['time_spend_company'].unique()

In [ ]:
data['time_spend_company'].value_counts()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot('time_spend_company', data = data)
plt.show()

In [ ]:
data['promotion_last_5years'].unique()

In [ ]:
data['promotion_last_5years'].value_counts()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot('promotion_last_5years', data = data)
plt.show()

In [ ]:
data['salary'].unique()

In [ ]:
data['salary'].value_counts()


In [ ]:
plt.figure(figsize=(15,6))
sns.countplot('salary', data = data, palette = 'hls')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
data = data.rename(columns = {'sales':'department'})

In [ ]:
data

In [ ]:
data['department']=np.where(data['department'] =='support','technical', data['department'])
data['department']=np.where(data['department'] =='IT','technical', data['department'])

In [ ]:
data

In [ ]:
cat_vars=['salary']
for var in cat_vars:
    cat_list='var'+'_'+var
cat_list = pd.get_dummies(data[var], prefix=var)
data1=data.join(cat_list)
data=data1

In [ ]:
salary_high=pd.get_dummies(data["salary"], drop_first=True)
salary_high.head(5)

In [ ]:
data2 = pd.concat([data,salary_high],axis=1)

In [ ]:
data2

In [ ]:
data

In [ ]:
data.drop(data.columns[[8]], axis=1, inplace=True)
data.columns.values


In [ ]:
data.drop(data.columns[[8, 9]], axis=1, inplace=True)
data.columns.values

In [ ]:
data

In [ ]:
cols=['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'salary_high', 'salary_low',
       'salary_medium']
X=data[cols]
y=data['left']

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=0)
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()

In [ ]:
from sklearn.metrics import accuracy_score
print('Logistic regression accuracy: {:.3f}'.format(accuracy_score(y_test,logreg.predict(X_test))))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
print('Random Forest Accuracy: {:.3f}'.format(accuracy_score(y_test,rf.predict(X_test))))

In [ ]:
y_pred = rf.predict(X_test)
from sklearn.metrics import confusion_matrix
import seaborn as sns
forest_cm = metrics.confusion_matrix(y_pred, y_test)
sns.heatmap(forest_cm, annot=True, fmt='.2f',xticklabels = ["Left", "Stayed"] , yticklabels = ["Left", "Stayed"])
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.title('Random Forest')

In [ ]:
logreg_y_pred = logreg.predict(X_test)
logreg_cm = metrics.confusion_matrix(logreg_y_pred, y_test)
sns.heatmap(logreg_cm, annot=True, fmt='.2f',
xticklabels = ["Left", "Stayed"] ,
yticklabels = ["Left", "Stayed"] )
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.title('Logistic Regression')

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
rf_roc_auc = roc_auc_score(y_test, rf.predict(X_test))
rf_fpr, rf_tpr, rf_thresholds = roc_curve(y_test, rf.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot(rf_fpr, rf_tpr, label='Random Forest (area = %0.2f)' % rf_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()